In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import json


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<style>.container { width:100% !important; }</style>

In [5]:
with open('data/data.json', 'r') as myfile:
    data=myfile.read()
    # parse file
    obj = json.loads(data)

# clean up icelandic letters from JSON
for key in obj[u'content_sent']:
    obj[u'content_sent'][key] = cleanMessage(obj[u'content_sent'][key])

for key in obj[u'content_recieved']:
    obj[u'content_recieved'][key] = cleanMessage(obj[u'content_recieved'][key])

# load the dataset to pandas dataframe
dataset = pd.DataFrame([obj[u'content_recieved'], obj[u'content_sent']]).T
dataset.columns = ['content_recieved', 'content_sent']

#dataset = pd.read_json(obj)
#dataset = dataset.drop(columns=["timestamp_send", "timestamp_recieved"], axis=1)
#dataset = dataset[:1000]

In [6]:
dataset.sample(10)

,content_recieved,content_sent
541,margir?,Jaaaaáá alveg
308,Þið!,jájá var heldur ekki spurning um það :) heyri ...
153,?,"yeahhh man, sucks!"
1051,kewl.,"hæbb, ertu með eitthvað geggjað tiki lag í hug..."
196,voru,njee 4
260,Til i thad,"næsari, þurfum að fá staðfest að við getum fen..."
150,What's up?,"was hoping you might do me a favour, need to g..."
701,mmm,mmmhhhhrrr
312,Jolin,jú hann fór aldrei í þá keppni.. óvart. sem er...
1214,Haha ok,ahhh the world class party ... no im right in ...


In [7]:
# pre-processing clean up

dataset.content_recieved=dataset.content_recieved.apply(lambda x: str(x).lower())
dataset.content_sent=dataset.content_sent.apply(lambda x: str(x).lower())

In [8]:
# Take the length as 50
dataset.content_recieved=dataset.content_recieved.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
dataset.content_sent=dataset.content_sent.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [9]:
exclude = set(string.punctuation)
dataset.content_recieved=dataset.content_recieved.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
dataset.content_sent=dataset.content_sent.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [10]:
remove_digits = str.maketrans('', '', digits)
dataset.content_recieved=dataset.content_recieved.apply(lambda x: x.translate(remove_digits))
dataset.content_sent=dataset.content_sent.apply(lambda x: x.translate(remove_digits))

In [11]:
dataset.content_recieved=dataset.content_recieved.apply(lambda x: cleanMessage(x))
dataset.conent_sent=dataset.content_sent.apply(lambda x: cleanMessage(x))

dataset.sample(5)

/Users/villi/Desktop/Projects/ChatBotAI/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,content_recieved,content_sent
387,heheh,ég var alveg farinn að halda að ég væri ekki a...
600,kominn,er að koma
695,nomnomnom,hey villtu vera partur af reykjavik pub crawl ...
105,eh,jebbs bara ekki of lengi þá skjótast þær af
647,mmm,you sent a gif from giphy


In [12]:
dataset.content_sent = dataset.content_sent.apply(lambda x : 'START_ '+ x + ' _END')
# TODO: kannski þarf að gera þetta eftir næsta skref

In [13]:
input_words = set()
target_words = set()

for eng in dataset.content_recieved:
    for word in eng.split():
        if word not in input_words:
            input_words.add(word)
    
for fr in dataset.content_sent:
    for word in fr.split():
        if word not in target_words:
            target_words.add(word)

#target_words = set(map(lambda x : 'START_ ' + x + ' _END', target_words))

In [14]:
lenght_list=[]
for l in dataset.content_recieved:
    lenght_list.append(len(l.split(' ')))
max_encoder_seq_length = np.max(lenght_list)


lenght_list=[]
for l in dataset.content_sent:
    lenght_list.append(len(l.split(' ')))
max_decoder_seq_length = np.max(lenght_list)

In [15]:

input_words_list = sorted(list(input_words))
target_words_list = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
# del all_eng_words, all_french_words

In [16]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words_list)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words_list)])

In [17]:
encoder_input_data = np.zeros(
    (len(dataset.content_recieved), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(dataset.content_sent), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(dataset.content_sent), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [18]:
for i, (input_text, target_text) in enumerate(zip(dataset.content_recieved, dataset.content_sent)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [19]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model


embedding_size = 50

Using TensorFlow backend.


In [20]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     34500       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     124050      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [23]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64,
          epochs=1,
          validation_split=0.05)

# Save model
model.save('saved_model.h5')

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Train on 1173 samples, validate on 62 samples
Epoch 1/1


  64/1173 [>.............................] - ETA: 1:00 - loss: 0.4615 - acc: 0.0000e+00

 128/1173 [==>...........................] - ETA: 38s - loss: 0.4684 - acc: 0.0020     

...

1024/1173 [=========================>....] - ETA: 3s - loss: 0.5113 - acc: 0.0074

1088/1173 [==========================>...] - ETA: 1s - loss: 0.5100 - acc: 0.0074

1152/1173 [============================>.] - ETA: 0s - loss: 0.5067 - acc: 0.0075

1173/1173 [==============================] - 28s 24ms/step - loss: 0.5087 - acc: 0.0075 - val_loss: 0.4441 - val_acc: 0.0082


/Users/villi/Desktop/Projects/ChatBotAI/venv/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [24]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          34500     
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 54,700
Trainable params: 54,700
Non-trainable params: 0
_________________________________________________________________


In [25]:

decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [47]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 11):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [45]:

for seq_index in [455, 329]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    print(type(input_seq))
    print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', dataset.content_sent[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

<class 'numpy.ndarray'>
[[576.   0.   0.]]
[[1.]]
-
Input sentence: 297    START_ what _END
Name: content_sent, dtype: object
Decoded sentence:  _END
<class 'numpy.ndarray'>
[[260.   0.   0.]]
[[1.]]
-
Input sentence: 183    START_ okeibb  pikka þig upp þá _END
Name: content_sent, dtype: object
Decoded sentence:  _END


In [4]:
def cleanMessage(message):
    message = message.replace("\u00c3\u00b0", "ð")
    message = message.replace("\u00c3\u00b0", "ð")

    message = message.replace("\u00c3\u00ad", "í")
    message = message.replace("\u00c3\u008d", "Í")

    message = message.replace("\u00c3\u00b3", "ó")
    message = message.replace("\u00c3\u0093", "Ó")

    message = message.replace("\u00c3\u00a6", "æ")

    message = message.replace("\u00c3\u00a9", "é")

    message = message.replace("\u00c3\u00ba", "ú")

    message = message.replace("\u00c3\u00be", "þ")
    message = message.replace("\u00c3\u009e", "Þ")

    message = message.replace("\u00c3\u00a1", "á")
    message = message.replace("\u00c2\u00b4", "á")

    message = message.replace("\u00c3\u00bd", "ý")
    message = message.replace("\u00c3\u009d", "ý")

    message = message.replace("\u00c3\u00b6", "ö")
    
    return message

In [48]:
w = "villi"
tmpArr = []
for t, word in enumerate(w.split()):
        tmpArr.append(input_token_index[word])

ndArr = np.array(tmpArr)
print(ndArr)
decoded_sentence = decode_sequence(ndArr)
print(decoded_sentence)
        


[592]
2
 _END
